In [1]:
import os
# import conllu
# import Levenshtein as lev
import numpy as np
import re
import tqdm
import json
from bs4 import BeautifulSoup
from collections import defaultdict

In [3]:
def levenshtein_distance(s1, s2):
    l_s1 = len(s1)
    l_s2 = len(s2)
    dist = np.zeros((l_s1 + 1, l_s2 + 1))

    # Distanta de la prefixul care incepe pe pozitia i din s1 la sirul vid
    for i in range(l_s1 + 1):
        dist[i, 0] = i
    for i in range(l_s2 + 1):
        dist[0, i] = i

    for i in range(1, l_s1 + 1):
        for j in range(1, l_s2 + 1):
            # daca ultimul caracter este acelasi
            if s1[i - 1] == s2[j - 1]:
                dist[i, j] = dist[i - 1, j - 1]
            else:
                deletion = dist[i - 1, j]
                insertion = dist[i, j - 1]
                substitution = dist[i - 1, j - 1]
                dist[i, j] = 1 + min(deletion, insertion, substitution)
    
    return dist[l_s1, l_s2]

In [4]:
file = open("./dataset/Tales.train.ro", "r", encoding="utf-8")
all_words_ro = set()
for line in file:
    line = line.strip()
    regex = re.compile(r"(\w+)")
    words = re.findall(regex, line)
    for word in words:
        all_words_ro.add(word)

In [5]:
file = open("dataset/Tales.train.rup", "r", encoding="utf-8")
all_words_aro = set()
for line in file:
    line = line.strip()
    regex = re.compile(r"(\w+)")
    words = re.findall(regex, line)
    for word in words:
        all_words_aro.add(word)

In [6]:
json_file_path = "candidates.json"

def get_candidates():
    candidates_dict = {}
    all_words_ro_list = list(all_words_ro)
    all_words_aro_list = list(all_words_aro)
    for word in tqdm.tqdm(all_words_aro_list, desc="Processing", unit="element"):
        best_candidates_dist = [levenshtein_distance(word, candidate) for candidate in all_words_ro_list]
        best_candidates_ind = np.argsort(best_candidates_dist)
        candidates_dict[word] = [all_words_ro_list[ind] for ind in best_candidates_ind[:10]]
    # print(candidates_dict)


    with open(json_file_path, 'w') as json_file:
        json.dump(candidates_dict, json_file)
    print(f"Dictionary saved to {json_file_path}")



îndoauă = unele (distanta mare) -- *Sunt unele deprinderi* - *Sănt îndoauă învețuri*\
îndoauă = in doua _minute_(distanta mica), rapid  -- *Tricură ași îndoauă minute*\
îndoauă ~ indata, in doua (gasite cu distanat Levenshtein)




In [7]:
with open(json_file_path, 'r') as json_file:
    candidates_dict_test = json.load(json_file)

print(candidates_dict_test['îndoauă'])

['îndată', 'înceapă', 'două', 'învață', 'înecată', 'întoarcă', 'încoace', 'Îndată', 'doară', 'înhață']


In [ ]:
# Aici am incercat sa parsez dictionarul Cunia

# html_file = open("Dictionar_Cunia.html", "r", encoding="utf-8")
# soup = BeautifulSoup(html_file, "html.parser")
# print(soup.prettify()[0:10000])
# def find_root_word(tag):
#     return tag.name == 'span' and tag.get('style') == 'text-decoration: none'
# def find_translation(tag):
#     return tag.name == 'font' and tag.get('style') == 'font-size: 10pt' and tag.get('size') == '2'
# def find_def_and_ex(tag):
#     return tag.name == 'font' and tag.get('face') == 'Times New Roman, serif'


In [ ]:
def_ex = soup.find_all(find_def_and_ex)
for el in def_ex[:100]:
    print(el.text)

In [9]:

"""
with open('dataset/Tales.train.rup', 'r', encoding='utf-8') as file:
    text_aro = file.read()

with open('dataset/Tales.train.ro', 'r', encoding='utf-8') as file:
    text_ro = file.read()


from nltk.tokenize import word_tokenize
words_dialect1 = word_tokenize(text_aro)
words_dialect2 = word_tokenize(text_ro)


from collections import Counter

co_occurrences = Counter()

for word1 in set(words_dialect1):
    for word2 in set(words_dialect2):
        if word1 in words_dialect1 and word2 in words_dialect2:
            co_occurrences[(word1, word2)] += 1


for (word1, word2), count in co_occurrences.most_common():
    print(f'Co-occurrence of "{word1}" and "{word2}": {count} times')

"""

'\nwith open(\'dataset/Tales.train.rup\', \'r\', encoding=\'utf-8\') as file:\n    text_aro = file.read()\n\nwith open(\'dataset/Tales.train.ro\', \'r\', encoding=\'utf-8\') as file:\n    text_ro = file.read()\n\n\nfrom nltk.tokenize import word_tokenize\nwords_dialect1 = word_tokenize(text_aro)\nwords_dialect2 = word_tokenize(text_ro)\n\n\nfrom collections import Counter\n\nco_occurrences = Counter()\n\nfor word1 in set(words_dialect1):\n    for word2 in set(words_dialect2):\n        if word1 in words_dialect1 and word2 in words_dialect2:\n            co_occurrences[(word1, word2)] += 1\n\n\nfor (word1, word2), count in co_occurrences.most_common():\n    print(f\'Co-occurrence of "{word1}" and "{word2}": {count} times\')\n\n'

In [ ]:
import json
import pandas as pd

reviewd_candidates = defaultdict(list)

json_file_path = "candidates.json"
with open(json_file_path, 'r') as json_file:
    candidates_dict_test = json.load(json_file)

print(candidates_dict_test['îndoauă'])

excel_path = "Papahagi.xls"
df = pd.read_excel(excel_path, header = None)

df.columns = ["POS", "aro", "ro", "origine", "IDK", "autor"]
df.head()

nr_neacoperite_de_dictionar = 0
for aro_word, values in tqdm.tqdm(candidates_dict_test.items()):
    if df[df["aro"] == aro_word].shape[0] == 0:
        nr_neacoperite_de_dictionar += 1
        continue

    rev = [val for val in values if val in df[df["aro"] == aro_word]["ro"].values.tolist()]
    if rev != []:
        reviewd_candidates[aro_word] = rev

reviewd_without_allignment = defaultdict(list)
for word, value in tqdm.tqdm(candidates_dict_test.items()):
    remaining = [val for val in value if word in df["ro"]]
    reviewd_without_allignment[word] = remaining



['îndată', 'înceapă', 'două', 'învață', 'înecată', 'întoarcă', 'încoace', 'Îndată', 'doară', 'înhață']


100%|██████████| 8356/8356 [00:00<00:00, 45965.52it/s]


In [14]:
print(reviewd_candidates.values())
print(len(reviewd_candidates.values()))
print(nr_neacoperite_de_dictionar)

print(len(reviewd_without_allignment.values()))

dict_values([['apus'], ['drept'], ['degeaba'], ['o'], ['bou'], ['încolo'], ['înăuntru'], ['vostru'], ['doi'], ['meu'], ['apoi'], ['din'], ['sub'], ['încolo'], ['domn'], ['înapoi', 'apoi'], ['încoa'], ['Bosnia'], ['în'], ['aproape'], ['semn'], ['întru'], ['putere'], ['lucru'], ['codru'], ['dacă'], ['lemn'], ['înainte'], ['voi'], ['noi'], ['suflet'], ['furnică'], ['seu'], ['la'], ['cu'], ['tu'], ['para'], ['herghelie'], ['tău'], ['curând'], ['cum'], ['nou'], ['ta'], ['zmeu'], ['ca'], ['untdelemn'], ['mai'], ['mort'], ['traistă'], ['larg'], ['greu'], ['de'], ['după'], ['rău'], ['care'], ['cal'], ['ieri'], ['întru'], ['ea'], ['cale'], ['singur'], ['apoi'], ['socru'], ['bun'], ['prin'], ['pe'], ['vânt'], ['albastru'], ['mea'], ['frate'], ['alaltăieri'], ['nu'], ['lung'], ['el'], ['lângă'], ['apoi'], ['întru'], ['armânesc'], ['rău'], ['iar'], ['gard'], ['contra'], ['când'], ['mult'], ['cuib'], ['mai'], ['că'], ['soi'], ['mormânt'], ['un'], ['trei'], ['pe'], ['acolo'], ['ars']])
94
8037
8356


#### Doar 94 dintre cuvintele din aromana pastreaza candidati in romana dupa verificarea daca acestia exista in dictionar(dintre cele procesate cu distanta Levenshtein).
#### 8037 dintre cuvintele care apar in textele in aromana nu se gasesc in dictionar
#### 8356 se gasesc in dictionar dar nu neaparat ca traducere a cuvantului corespunzator

In [13]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
with open('./dataset/Tales.train.ro', 'r', encoding='utf-8') as file:
    ro_text = file.read()

with open('./dataset/Tales.train.rup', 'r', encoding='utf-8') as file:
    aro_text = file.read()

ro_tokens = word_tokenize(ro_text.lower())
aro_tokens = word_tokenize(aro_text.lower())
vectorizer = CountVectorizer()


# co_occurrences_matrix = []
# for aro_token in aro_tokens:
#     for ro_token in ro_tokens:
#         co_occurrences_matrix.append((aro_token, ro_token))

# X = vectorizer.fit_transform(co_occurrences_matrix)

        

# similarity_scores = cosine_similarity(X)
# print(similarity_scores)
# threshold = 0.6

# aligned_indices = np.where(similarity_scores >= threshold)


In [24]:
direct_translation = dict()
for word in tqdm.tqdm(aro_tokens):
    if word in df["aro"].values.tolist():
        # print(word)
        # print(df[df["aro"] == word]["ro"].values.tolist())
        direct_translation[word] = df[df["aro"] == word]["ro"].values.tolist()
        

100%|██████████| 66268/66268 [00:40<00:00, 1620.65it/s]


In [39]:
print(len(direct_translation))
print(len(aro_tokens))

299
66268


Codul de mai jos incearca sa gaseasca o aliniere intre cuvinte folosind un window de 4 cuvinte in ambele parti si pozitia unui cuvant.\
Pentru toate cuvintele dureaza ~ 6 ore :| + consuma foarte multa memorie, salvand fiecare cuvant din text care apare la o distanta de +- 4 pentru fiecare alt cuvant.\
Repeta de prea multe ori unele cuvinte din ferestre.

In [32]:
from collections import defaultdict
import numpy as np


context_window = 4

def get_context_words(tokens, index, window):
    start = max(0, index - window)
    end = min(len(tokens), index + window + 1)
    return tokens[start:index] + tokens[index + 1:end]


ro_context_pairs = [(word, context_word) 
                    for index, word in enumerate(ro_tokens)
                    for context_word in get_context_words(ro_tokens, index, context_window)]
aro_context_pairs = [(word, context_word) 
                     for index, word in enumerate(aro_tokens)
                     for context_word in get_context_words(aro_tokens, index, context_window)]


word_co_occurrence = defaultdict(lambda: defaultdict(int))
for (ro_word, ro_context) in tqdm.tqdm(ro_context_pairs[100:120]):
    print("ro_word", ro_word)
    print("ro_context", ro_context)
    for (aro_word, aro_context) in aro_context_pairs:
        # print("ro_word", ro_word)
        # print("ro_context", ro_context)
        # print("aro_word", aro_word)
        # print("aro_context", aro_context)
        if ro_context == aro_context:
            word_co_occurrence[ro_word][aro_word] += 1




 25%|██▌       | 5/20 [00:00<00:00, 41.54it/s]

ro_word ,
ro_context despre
ro_word ,
ro_context ele
ro_word ori
ro_context câștigi
ro_word ori
ro_context doar
ro_word ori
ro_context citind
ro_word ori
ro_context ,
ro_word ori
ro_context vorbindu-ți-se
ro_word ori
ro_context despre
ro_word ori
ro_context ele


 70%|███████   | 14/20 [00:00<00:00, 39.37it/s]

ro_word ori
ro_context .
ro_word vorbindu-ți-se
ro_context doar
ro_word vorbindu-ți-se
ro_context citind
ro_word vorbindu-ți-se
ro_context ,
ro_word vorbindu-ți-se
ro_context ori
ro_word vorbindu-ți-se
ro_context despre
ro_word vorbindu-ți-se
ro_context ele
ro_word vorbindu-ți-se
ro_context .
ro_word vorbindu-ți-se
ro_context dar
ro_word despre
ro_context citind


100%|██████████| 20/20 [00:00<00:00, 39.31it/s]

ro_word despre
ro_context ,


In [33]:
print(word_co_occurrence.keys())

dict_keys(['ori', 'vorbindu-ți-se', 'despre'])


In [35]:
print(word_co_occurrence["despre"])

defaultdict(<class 'int'>, {'„': 132, 'sănt': 1, 'îndoauă': 3, 'învețuri': 1, 'cari': 175, 'si': 264, 'amintă': 5, 'mași': 61, 'cu': 660, 'ghivăsirea': 1, 'i': 940, 'avdzărea': 1, '.': 1188, 'și': 1097, 'alte': 19, 'lucri': 5, 'nu': 562, 'intră': 32, 'tu': 506, 'avdzăm': 1, 'multe': 33, 'videm': 2, 'ma': 393, 'tute': 77, 'aiste': 11, '-': 1411, 'țiva': 74, 'știm': 2, 'pînă': 118, 's-nu': 84, 'băgăm': 3, 'măna': 1, 'zboară': 6, 'voi': 25, 'ca': 401, 's-dzîc': 4, 'eu': 53, 'că': 637, 'teoria': 3, 'easte': 84, 'bună': 26, ',': 2808, 'di': 1223, 'ori': 45, 'ama': 1, 'căte': 1, 'unoară': 9, 'armăne': 1, 'un': 346, 'auș': 16, 'minte': 18, 'cătră-ndoi': 1, 'tiniri': 3, 'ninga': 31, 'avea': 61, 'ișetă': 1, 'dit': 72, 'școală': 2, 'aveau': 1, 'ideia': 1, 'știe': 9, 'năși': 8, '...': 199, 'tut': 65, 'ți-azboairă': 2, 's-măcă': 4, '!': 693, 's-lă': 16, 'va': 168, 'dzîșu': 1, 'pîn': 24, 'aua': 10, 'ascultaț': 2, 'ți': 126, 'aleaptă': 4, 'părăvulie': 2, 'alăgat': 1, 'căt': 2, 'vreț': 3, 'își': 66, 

Aici am incercat sa creez un dataset paralel cu propozitii/cuvinte si sa aplic un IBM Model pe el in speranta o sa mai obtinem ceva alinieri\
pe care sa le comparam cu restul de alinieri pe care le avem

In [74]:
parallel_corpus = []
for key, value in direct_translation.items():
    for word in value[:1]:
        parallel_corpus.append((word, key))

print(parallel_corpus)

[('care', 'cari'), ("s'", 'si'), ('cu', 'cu'), ('sau', 'i'), ('nu', 'nu'), ('în', 'tu'), ('al, a, ai, ale, ', 'a'), ('al', 'li'), ('ci', 'ma'), ('ca', 'ca'), ('bunăoară', 'pare'), ('că', 'că'), ('voi', 'voi'), ('dacă', 'di'), ('capricii', 'ori'), ('dacă', 'ama'), ('foarte', 'multu'), ('chiar', 'ninga'), ('aici', 'aua'), ('în', 'în'), ('când', 'cănd'), ('contra', 'pi'), ('afacere', 'lucru'), ('din', 'din'), ('patru ', 'patru'), ('corabie', 'nai'), ('încă ', 'cama'), ('în', 'tru'), ('de ', 'ti'), ('ea', 'ea'), ('ca', 'ta'), ('aşadar', 'cara'), ('acolo', 'aclo'), ('la', 'la'), ('unde', 'iu'), ('o', 'u'), ('dar', 'e'), ('vostru', 'vostru'), ('fi, v', 'escu'), ('iată', 'ia'), ('nici', 'ni'), ('aşadar', 'de'), ('om', 'suflitu'), ('noi', 'noi'), ('părinte', 'afendi'), ('de loc', 'dip'), ('care', 'cai'), ('trei', 'trei'), ('iar', 'iar'), ('un', 'unu'), ('oare', 'au'), ('domnul meu', 'efendim'), ('în', 'pri'), ('trebuie ', 'vrea'), ('eu', 'mini'), ('chema, v', 'grescu'), ('de-a', 'na'), ('absen

In [91]:
import string
def create_parallel_sent(ro_tokens, aro_tokens, index, window):
    start_ro = max(0, index - window)
    end_ro = min(len(ro_tokens), index + window + 1)

    start_aro = max(0, index - window)
    end_aro = min(len(aro_tokens), index + window + 1)

    parallel_words = (aro_tokens[start_aro:index + 1] + aro_tokens[index + 1:end_aro], ro_tokens[start_ro:index + 1] + ro_tokens[index + 1:end_ro])
    
    punctuations = string.punctuation + "„"
    parallel_sent = (" ".join([word for word in parallel_words[0] if word not in punctuations]), " ".join([word for word in parallel_words[1] if word not in punctuations]))

    return parallel_sent



In [92]:
parallel_corpus = []
for i in range(100):
    print(create_parallel_sent(ro_tokens, aro_tokens, i, 4))
    parallel_corpus.append(create_parallel_sent(ro_tokens, aro_tokens, i, 4))

('teoria și practica sănt', 'teoria și practica sunt')
('teoria și practica sănt îndoauă', 'teoria și practica sunt unele')
('teoria și practica sănt îndoauă învețuri', 'teoria și practica sunt unele deprinderi')
('teoria și practica sănt îndoauă învețuri', 'teoria și practica sunt unele deprinderi pe')
('teoria și practica sănt îndoauă învețuri cari', 'teoria și practica sunt unele deprinderi pe care')
('și practica sănt îndoauă învețuri cari si', 'și practica sunt unele deprinderi pe care le')
('practica sănt îndoauă învețuri cari si amintă', 'practica sunt unele deprinderi pe care le câștigi')
('sănt îndoauă învețuri cari si amintă mași', 'sunt unele deprinderi pe care le câștigi doar')
('sănt îndoauă învețuri cari si amintă mași cu', 'sunt unele deprinderi pe care le câștigi doar citind')
('îndoauă învețuri cari si amintă mași cu ghivăsirea', 'unele deprinderi pe care le câștigi doar citind')
('învețuri cari si amintă mași cu ghivăsirea', 'deprinderi pe care le câștigi doar citind 

In [108]:
import nltk
from nltk.translate import AlignedSent, IBMModel1

aligned_text = []
for pair in parallel_corpus:
    src_sent, trg_sent = pair
    src_tokens = src_sent.split()
    trg_tokens = trg_sent.split()
    aligned_text.append(AlignedSent(trg_tokens, src_tokens))

ibm_model = IBMModel1(aligned_text, 15)  


def translate(src_sentence):
    src_tokens = src_sentence.split()
    trg_tokens = [''] * len(src_tokens)  
    for i, src_word in enumerate(src_tokens):
        max_prob = 0
        for trg_word in ibm_model.translation_table[src_word]:
            prob = ibm_model.translation_table[src_word][trg_word]
            if prob > max_prob:
                max_prob = prob
                trg_tokens[i] = trg_word
    return ' '.join(trg_tokens)


ls = ["lucri cari nu", 'îndoauă alte', "armăne", "ghivăsirea i cu avdzărea"]
src_sentence = "ghivăsirea i cu avdzărea"
translated_sentence = translate(src_sentence)
print(f'Translation of "{src_sentence}": "{translated_sentence}"')


Translation of "ghivăsirea i cu avdzărea": "  ș-cu "


In [82]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
